<a href="https://colab.research.google.com/gist/teruyuki-yamasaki/e2129a0efbd7d9ff2954f903072309b7/rl_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**強化学習の基本的なモデル**

強化学習は、所与の環境の下でエージェントが行動パターンを最適化するという問題を扱う.　エージェントは方策$\pi$に基いて行動$a$を決定し環境に対して働きかける.　すると環境の状態$s$が変化しその変化に応じてエージェントは報酬$r$を取得する.　ここで報酬とは、エージェントが環境に及ぼした行動の結果の（エージェントにとっての）良し悪しについての情報を一元化した実数値である.　エージェントは得られる報酬の総額を最大化するような方策の探索を行う．これが強化学習の基本的なモデルになる.　なお、強化学習では、問題を扱いやすくするために、以下のようなことを前提とする：
- 環境モデルのマルコフ性：時点$t+1$の状態$s'$は時点$t$の状態$s$と行動$a$にのみ依存し、それより過去の情報には依存しない性質を有する.
$$Pr\{S_{t+1} = s' | S_0, A_0, S_1, A_1, \cdots, S_{t-1}, A_{t-1},S_t, A_t\} 
= Pr\{S_{t+1} = s' | S_t, A_t\}$$
※時点$t$より過去の情報は$S_t$に組み込まれていると捉えることができる.

- 状態$s$の下で行動$a$がなされたときに環境が次状態$s'$に遷移する過程がマルコフ確率過程によって決まる：
$$p(s' | s, a) = Pr\{S_{t+1}=s' | S_t = s, A_t = a\}$$

- 状態$s$の下で行動$a$がなされ環境が次状態$s'$に遷移したときにエージェントが得る報酬$r$もマルコフ確率過程により与えられ、その期待値はステップ$(s,a,s')$毎に決められる：
$$
\begin{eqnarray}
p(r|s,a,s') &=& Pr\{R_{t+1} = r | S_t=s, A_t=a, S_{t+1} = s'\} \\
r_{s s'}^{a} &=& \sum_r r Pr\{R_{t+1} = r | S_t = s, A_t = a, S_{t+1} = s'\} 
\\ &=& E[R_{t+1} = r | S_t = s, A_t = a, S_{t+1} = s']
\end{eqnarray}
$$

- 方策$\pi$：エージェントが状態$s$の下で行動$a$を選択する過程もマルコフ確率過程：
$$\pi (a | s) = Pr\{A_t = a | S_t = s\}$$



**価値関数の定義**

早速、強化学習における最適化の指標となる報酬の総額について定義していく.

環境モデル$p(s'|s,a)$、$p(r|s,a,s')$、 方策$\pi(a|s)$の下で、ある状態・行動・報酬系列(history)
$$
\begin{align}
H = \{(S_t, A_t, R_{t+1})\}_{t=0}^{\infty}
= \{S_0, A_0, R_1, S_1, A_1, R_2, \cdots, S_t, A_t, R_{t+1}, \cdots\}
\end{align}
$$
を取得したとする. \\
このとき、状態$S_t=s$以降に得られた報酬の時点tにおける現在価値の総和は
$$
\begin{align}
G_t = R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + \cdots = \sum_{k=1}^\infty \gamma^{k-1}R_{t+k}
\end{align}
$$
のように、$\gamma$を割引率とする割引報酬和として与えられる.これを時点$t$以降の収益という. \\

(※終端状態T(Terminal)が存在する場合は、$R_{T+k} = 0$ $(k = 1,2,\cdots)$として
$$
\begin{align}
G_t = \sum_{k=1}^{T-t} \gamma^{k-1}R_{t+k} + \sum_{k=T-t+1}^\infty \gamma^{k-1}R_{t+k} = \sum_{k=1}^{T-t} \gamma^{k-1}R_{t+k}
\end{align}
$$
とでき、収益の元の定義に内包されていることがわかる.)

この定義から、収益は、所与の環境において方策$\pi$に従って行動していったときに取得される報酬のサンプル値から決まる値、すなわち$\pi$に依存した値であるといえる.

よって、方策$\pi$の下での各状態$s$の価値は、所与の環境において方策$\pi$に従っていったときに状態$S_t = s$以降に得られる収益$G_t$の期待値

$$
\begin{align}
v_\pi(s) = E_\pi [G_t | S_t = s]
\end{align}
$$

として定義される.　

以上より、強化学習における最適化問題は、「すべての状態$s \in S$において価値関数$v_\pi(s)$が最大となるような方策$\pi(a|s)$を求める」という問題になる.

**ベルマン方程式の導出**

いま定義した価値関数$v_\pi(s)$を扱いやすくするために、ベルマン方程式を導出していく.

再帰性の利用：

まず、価値関数$v_\pi(s)$を求めるために、
$$
\begin{eqnarray}
G_t 
&=& R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + \cdots 
\\ &=& R_{t+1} + \gamma \underbrace{(R_{t+2} + \gamma R_{t+3} + \cdots)}_{G_{t+1}}
\\ &=& R_{t+1} + \gamma G_{t+1}
\end{eqnarray}
$$
という再帰性を利用して変形していく.　期待値の線形性から、価値関数は、即時報酬についての項と次時点以降の収益の期待値についての項の二項に分解できる：
$$
\begin{eqnarray}
E_\pi [G_t | S_t = s] 
& = & E_\pi [R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + \cdots|S_t = s]
\\ & = & E_\pi [R_{t+1} + \gamma (R_{t+2} + \gamma R_{t+3} + \cdots)|S_t = s]
\\ & = & E_\pi [R_{t+1} + \gamma G_{t+1} | S_t = s]
\\ & = & E_\pi [R_{t+1}|S_t =s] + \gamma E_\pi [G_{t+1} | S_t = s]
\end{eqnarray}
$$
以下で、最終行の第一項と第二項をそれぞれみていく.

第一項：

まず、第一項について.
$$E_\pi [R_{t+1}|S_t =s] = \sum_{a \in A} \underbrace{ Pr\{A_t=a|S_t = s\} }_{\pi(a|s)} \underbrace{E_\pi[R_{t+1}|S_t=s, A_t=a]}_{(*)}$$

$$
\begin{eqnarray}
(*)& = & \sum_{s'\in S}Pr\{S_{t+1}=s'|S_t=s,A_t=a\}E_\pi[R_{t+1}|S_t=s,A_t=a,S_{t+1}=s']
\\ & = & \sum_{s'\in S}p(s'|s,a)E[R_{t+1}|S_t=s,A_t=a,S_{t+1}=s'] 
\\ & = & \sum_{s'\in S}p(s'|s,a) r_{s s'}^{a}
\end{eqnarray}
$$

$$\therefore E_\pi [R_{t+1}|S_t =s] = \sum_{a \in A} \pi(a|s)\sum_{s'\in S}p(s'|s,a) r_{s s'}^{a} = r_s^\pi$$

意味としてはそのままで、方策$\pi$の下で状態$s$から1ステップ経たときに得る報酬の期待値を表している.

第二項：

次に、第二項について.
$$
E_\pi [G_{t+1} | S_t = s] = \sum_{a \in A} \underbrace{ Pr\{A_t=a|S_t = s\} }_{\pi(a|s)} \underbrace{E_\pi[G_{t+1}|S_t=s, A_t=a]}_{(**)}
$$

$$
(**) =  \sum_{s' \in S} \underbrace{Pr\{S_{t+1}=s'|S_t=s,A_t=a\}}_{p(s'|s,a)}\underbrace{E_\pi[G_{t+1}|S_t=s, A_t=a, S_{t+1}=s']}_{E_\pi[G_{t+1}|S_{t+1}=s']= E_\pi[G_t|S_t=s']=v_\pi(s')}
$$

$$
\begin{eqnarray}
\therefore E_\pi [G_{t+1} | S_t = s]  
&=& \sum_{a \in A} \pi(a|s)\sum_{s' \in S} p(s'|s,a)v_\pi(s')
\\ &=& \sum_{s' \in S} \underbrace{\left(\sum_{a \in A}\pi(s'|s,a)p(s'|s,a)\right)}_{P_{ss'}^\pi}v_\pi(s') 
\\ &=& \sum_{s' \in S} P_{ss'}^\pi v_\pi(s')
\end{eqnarray}
$$

$P_{ss'}^\pi$は、方策$\pi$に従ったときに状態$s$から$s'$に遷移する確率である.　この第二項が意味するのは、次状態の価値関数の期待値（次状態以降に得られる収益の期待値の期待値）である.

追記：
$$
P_{ss'}^\pi = P^\pi(s'|s) 
$$
と表記するのが良い.

**価値関数が従うベルマン方程式：**

以上をまとめると、価値関数$v_\pi(s)$は、1ステップ刻んだときに得られる即時報酬の期待値$r_s^\pi$と、次状態の価値関数の期待値$\sum_{s'\in S}P_{ss'}^\pi v_\pi(s')$の現在価値($\gamma$倍)の和として展開でき、次のようにまとめられる：
$$
\begin{eqnarray}
v_\pi(s)
&=&  r_s^\pi + \gamma \sum_{s' \in S} P_{ss'}^\pi v_\pi(s')
\\ &=& \sum_{a \in A} \pi(a|s)\sum_{s'\in S}p(s'|s,a) r_{s s'}^{a} + \gamma \sum_{a \in A} \pi(a|s)\sum_{s' \in S} p(s'|s,a)v_\pi(s') 
\\ &=& \sum_{a \in A} \pi(a|s) \sum_{s' \in S}p(s'|s,a)(r_{ss'}^a + \gamma v_\pi(s'))
\\ 
\end{eqnarray}
$$
これがすべての$s \in S$について成り立っている. 


**価値関数のベルマン方程式のベクトル方程式**：

以上により、
$$
\begin{eqnarray}
[v^\pi]_s &=& v_\pi(s) 
\\ [R^\pi]_s &=& r_s^\pi
\\ [P^\pi]_{ss'} &=& P_{ss'}^\pi
\end{eqnarray}
$$
となるような、方策$\pi$のもとでの価値観数ベクトル$v^\pi$、報酬ベクトル$R^\pi$、状態遷移確率行列$P^\pi$を用いて、すべての状態$s \in S$のベルマン方程式をまとめて
$$
\begin{align}
v^\pi = R^\pi + \gamma P^\pi v^\pi 
\end{align}
$$
と表せる.　これは
$$
\begin{align}
v^\pi = (I - \gamma P^\pi)^{-1} R^\pi
\end{align}
$$
と解くことができる.　


**行動価値関数の導入**：

方策を改善するためには、各状態における各行動の良し悪しを評価する指標を使う必要がある.
そこで、方策$\pi$の下で状態$s \in S$のときに各行動$a \in A$を取ったときの価値を表す行動価値関数を
$$
\begin{align}
q_\pi(s,a) = E_\pi[G_t|S_t=S,A_t=a]
\end{align}
$$

と定義する.　

この定義より、価値関数$v_\pi(s)$は、状態$s$において方策$\pi(a|s)$に従って行動選択した際の行動価値関数$q_\pi(s,a)$の期待値となっていることがわかる.

$$
\begin{align}
v_\pi(s) = \sum_{a \in A} \pi(a|s) q_\pi(s,a)
\end{align}
$$


**行動価値関数が従うベルマン方程式の導出**：

状態価値関数のときと同様の変形を行って

$$
\begin{eqnarray}
E_\pi[G_t|S_t=s,A_t=a] 
&=& E_\pi[R_{t+1}|S_t=S,A_t=a] + \gamma E_\pi[G_{t+1}|S_t=S,A_t=a]
\\ &=& \sum_{s' \in S} p(s'|s,a) r_{ss'}^a + \gamma \sum_{s' \in S} p(s'|s,a)v_\pi(s')
\end{eqnarray}
$$

とできる.

すなわち、行動価値関数は状態価値関数を用いて
$$
\begin{align}
q_\pi(s,a) = \sum_{s' \in S} p(s'|s,a) \left(r_{ss'}^a + \gamma v_\pi(s')\right)
\end{align}
$$
と表せる.　これが行動価値関数が従うベルマン方程式である.

**方策反復法：greedyな方策改善**


方策$\pi$の下での各状態$s \in S$における最適行動$a_{\pi,s}^*$は、各状態行動対(s,a)の行動価値関数$q_\pi(s,a)$を比較して
$$
\begin{align}
a_{\pi,s}^* = \mbox{argmax}_{a \in A} q_\pi(s,a)
\end{align}
$$
により決定される.
これに基いて、greedyに方策を更新する：
$$
\begin{align}
\pi_{old}(a|s) \rightarrow \pi_{new}(a|s) = \left\{ 
  \begin{array}{cc}
  1 / |A_{\pi_{old}}^*(s)|, & a \in A_{\pi_{old}}^*(s) \\ 
  0, & \mbox{otherwise}
  \end{array} \right.
\end{align}
$$

ただし、$A_{\pi_{old}}^*(s)$は更新前の方策$\pi_{old}$の下での状態$s$における最適行動$a_{\pi_{old},s}^*$の集合であり、$A(s)$の部分集合:
$$A_{\pi_{old}}^*(s) = \left\{ a^* \in A(s)| a^* = \mbox{argmax}_a q_{\pi_{old}}(s,a)\right\}$$

以上のように更新した新たな方策$\pi_{new}$の下で、また新たに価値関数の導出、行動価値関数の導出、方策更新を行う.　
この操作を繰り返すと方策は更新されなくなり、最適方策$\pi_*$に収束する.
※最適方策への収束の証明については別途参照.

**最適ベルマン方程式の導出：**

最適状態の定義：
$$
\begin{align}
\pi_* = \mbox{argmax}_\pi v_\pi(s),\quad \mbox{for all} \ s \in S
\end{align}
$$
のとき、方策$\pi_*$は最適方策である.

また、最適価値関数、最適行動価値関数はそれぞれ
$$
\begin{eqnarray}
v_*(s) &=& \mbox{max}_\pi v_\pi(s) \\
q_*(s,a) &=& \mbox{max}_\pi q_\pi(s,a)
\end{eqnarray}
$$
として定義される.

最適ベルマン方程式の導出：

価値関数と行動価値関数の関係式にこれらを代入すると
$$
\begin{eqnarray}
v_*(s) 
&=& \sum_{a \in A}\pi_*(a|s) q_*(s,a)
\\ &=& \sum_{a_{\pi_*, s}^* \in A(s)}\pi_*(a_{\pi_*,s}^*|s) q_*(s,a_{\pi_*,s}^*) 
\\ &=& |A^*(s)|\frac{1}{|A^*(s)|}\mbox{max}_{a}q_*(s,a) = \mbox{max}_a q_*(s,a) 
\end{eqnarray}
$$
すなわち、
$$
\begin{align}
v_*(s) = \mbox{max}_a r_\pi(s,a) 
\end{align}
$$
となる.　ただし、
$$
\begin{align}
a_{\pi_*,s}^*  =  \mbox{argmax}_s q_*(s,a)
\end{align}
$$
また、行動価値関数のベルマン方程式より
$$
\begin{align}
q_*(s,a) = \sum_{s' \in S}p(s'|s,a)\left(r_{ss'}^a + \gamma v_*(s') \right) 
\end{align}
$$
が得られる.　

以上をまとめると、最適状態において
$$
\begin{eqnarray}
q_*(s,a) & = & \sum_{s' \in S}p(s'|s,a)\left(r_{ss'}^a + \gamma v_*(s') \right) \\
v_*(s)  & = & \mbox{max}_a q_*(s,a) \\
a_{\pi_*,s}^* & = & \mbox{argmax}_a q_*(s,a)
\end{eqnarray}
$$
が成り立っている．

**価値反復法：**

方策反復法においてベルマン方程式の解析解を求める計算コストが大きいときは、代わりに、最適ベルマン方程式の形のをした漸化式に適当な初期値を代入して逐次計算を行い極限を求めることによって最適状態を求める.

$$
\begin{eqnarray}
q_{k+1}(s,a) & = & \sum_{s' \in S}p(s'|s,a)\left(r_{ss'}^a + \gamma v_k(s') \right) \\
v_{k+1}(s)  & = & \mbox{max}_a q_{k+1}(s,a) \\
a_{\pi_k,s}^* & = & \mbox{argmax}_a q_{k+1}(s,a) \\
\end{eqnarray}
$$


環境モデルが既知のときは、以上のように、状態・行動系列の全ての分岐について期待値計算を行うことができ、それに基づいて方策反復法あるいは価値反復法を用いて最適方策を決定することができることをみた（動的計画法： dynamic programming）.　次に、環境モデルが未知の場合に方策を最適化する方法について議論する.　

環境モデルが未知のときは、全条件分岐について厳密に期待値計算を行うことはできないため、動的計画法のような方法で最適方策を求めることはできない.　そのため、エージェントは、モデル未知の所与の環境において繰り返し行動選択と報酬取得を繰り返し、状態・行動・報酬系列をサンプリングしつつ探索的に最適方策を学習する必要がある.　


**モンテカルロ法(Monte Carlo, MC)**:

エピソードを最初から最後まで走らせ、得られた状態・行動・報酬系列から、各状態・行動に対する収益$G_t$のサンプリングして価値関数または行動価値関数を推定する方法.　教科書では$V$を推定しているが、ここでは$Q$を推定する.
アルゴリズムは以下の通り：
 - 各状態行動対の行動価値関数の推定値$Q(s,a)$を初期化する.
 - 未知の環境の下でエピソードをサンプリング: 
 $$\{S_0, A_0, R_1, S_1, A_1, R_2, \cdots, S_{T-1}, A_{T-1}, R_{T},S_T\}$$
 - 得られた系列から、各エピソードにおける各時点t以降に得られる収益$G_t$を求める.
 $$G_t = R_{t+1} + \gamma R_{t+2} + \cdots + \gamma^{T-t-1} R_T$$
 - 各時点tでのステップごとに、全ての状態行動対$(s,a)$について
 $$
 \begin{array}{ll}
 N_{t+1}(s,a) = N_t(s,a) + 1(S_t=s,A_t=a) \\
 Q_{t+1}(s,a) = \frac{1}{N_{t+1}(s,a)}\sum_{k=0}^t G_k 1(S_k=s, A_k=a)
 \end{array}
 $$
 と更新していく.　
 - ひとつのエピソードについて更新が終わったら、また別のエピソードをサンプリングして同じことを繰り返す.
 - 各々の$G_t$は$q_\pi(s,a)=E_\pi[G_t|S_t=s, A_t = a]$を期待値とする確率変数だから、上記のような更新を繰り返して、$Q_\infty(s,a) \rightarrow q_\pi(s,a)$により行動価値関数が求まることになる.
 
 ※every visitとevery first visitの区別があるようだが、ここでは扱わない.


- ここで、$Q$の更新式の右辺は
 $$
 \begin{eqnarray}
 \frac{1}{N_{t+1}(s,a)}\sum_{k=0}^t G_k 1(S_k=s, A_k=a)
 &=&  \frac{N_{t+1}(s,a) - 1(S_t=s,A_t=a)}{N_{t+1}(s,a)}\frac{1}{N_{t}(s,a)}\sum_{k=0}^{t-1} G_k 1(S_k=s, A_k=a) + \frac{1}{N_{t+1}(s,a)}G_t 1(S_t=s,A_t=a)
 \\ &=& \left( 1 - \frac{1(S_t=s,A_t=a)}{N_{t+1}(s,a)}\right)Q_t(s,a) +\frac{1}{N_{t+1}(s,a)} G_t 1(S_t=s,A_t=a) 
 \\ &=& Q_t(s,a) + \frac{1}{N_{t+1}(s,a)}\left(G_t - Q_t(s,a)\right)1(S_t=s,A_t=a)
 \end{eqnarray}
 $$
と変形できるので、$Q$の更新式は
 $$
 Q_{t+1}(s,a) = Q_t(s,a) + \frac{1}{N_{t+1}(s,a)}\left(G_t - Q_t(s,a)\right)1(S_t=s,A_t=a)
 $$
 と表すこともできる.
- このことから、上述のように各ステップ毎に全ての状態行動対$(s,a)$について更新するのではなく、時点$t$で$(S_t,A_t)=(s,a)$となった状態行動対についてだけ
 $$
 \begin{array}{ll}
 N(s,a) \leftarrow N(s,a) + 1 & \\
 \alpha_{s,a} \leftarrow 1 / N(s,a) & \\
 Q(s,a) \leftarrow Q(s,a) + \alpha_{s,a} \left(G_t - Q(s,a)\right) & 
 \end{array}
 $$
 となるように情報を更新する方法に置き換えられる.
 - このアルゴリズムは、「真の値$q_\pi(s,a)$のサンプル値$G_t$を得る度に、推定値$Q(s,a)$が目標値（のサンプル）$G_t$に近づくように重み$\alpha_{s,a}$分だけ推定値$Q(s,a)$を更新する」ものと解釈でき、機械学習っぽい形式.
 - 各々の$G_t$は$q_\pi(s,a)$を期待値として環境モデル(未知)と方策$\pi$に従って分布する確率変数だから、そのときそのときの$G_t$に向かって重み$\alpha$分だけ$Q(s,a)$を近づけるという操作を繰り返すことで、長期的視点に立てば、$Q(s,a)$は$G_t$の期待値$q_\pi(s,a)$に近づいていくと考えられる.
 - この更新の仕方だと、ステップ$t$以前の情報は現在$t$の$N$、$Q$に上書きされていくので、最初の更新方法のように、過去の$S_t$や$A_t$の履歴を保持し更新のたびに毎回参照するような必要がない.

MCの問題点:
- $G_t = R_{t+1} + \gamma R_{t+2} + \cdots + \gamma^{T-t+1} R_T$を目標値として使うため、
 - 終端状態までの系列がサンプリングできるまで更新できない.
 - 必ず各エピソードが終端状態に達するモデルにしか適用できない.　


**TD学習**：

- 1 step収益$G_t^{(1)}$を目標値とした学習手法：
$$G_t^{(1)} = R_{t+1} + \gamma Q(S_{t+1},A_{t+1})$$

- この目標値は、本来の目標値（のサンプル値）
$$\begin{eqnarray}
G_t &=& R_{t+1} + \gamma R_{t+2} + \cdots + \gamma^{T-t+1} R_T
\\ &=& R_{t+1} + \gamma G_{t+1}
\end{eqnarray}$$
のうち、
 - 状態行動対$(S_t,A_t)$からの1ステップ先の即時報酬$R_{t+1}$(と$S_{t+1}$)だけを実際にサンプリング
  - 残りの$G_{t+1}$は、$Q_t(S_{t+1},A_{t+1})$(現状の推定値$Q_t$を使って推定した、現状の方策の下での状態$S_{t+1}$以降の収益の期待値)によって代替
  - $A_{t+1}$は、方策$\pi$に従ってサンプリング(SARSA)　または　$A_{t+1} = \mbox{argmax}_{a'} Q(S_{t+1},a')$ により決定(Q学習) \\

- 各ステップごとに、全ての状態行動対$(s,a)$について次のように$Q$関数を更新する：

$$
\delta_{t+1} = R_{t+1} + \gamma Q_t(S_{t+1},A_{t+1}) - Q_t(S_t,A_t) \\
Q_{t+1}(s,a) = Q_t(s,a) +  \alpha \delta_{t+1}1(S_t=s, A_t=a)
$$

- このときの目標値と推定値の誤差$\delta_{t+1}$を1-stepTD誤差(TD：temporal differnce)と呼ぶ.

- 更新のために必要なサンプリングは1ステップのみなのでオンライン学習が可能
- 更新ごとに行うサンプリングは1ステップ分のみであるとはいえ、$R_{t+1}$、$S_{t+1}$は、未知の環境モデルの確率変数を与えてくれるサンプル値である.　よって、更新を繰り返せば、$Q(s,a)$には、環境と行動の確率的な相互作用についての情報が蓄積されていく.
- しかし、やはり、サンプリングが1step分だけではbiasが大きくなることが問題である.

**n-step TD学習:**

- MCと1stepTDの間を取り持つ手法
 - 1 step TDのサンプリング不足を補う
 - MCのエピソード全体のサンプリングができてはじめて更新ができるという制約を緩和
- n-stepTD学習の目標値は、n-step収益：

$$\begin{eqnarray}
G_t^{(n)} 
&=& R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + \cdots + \gamma^{n-1}r_{t+n} + \gamma^n Q_t(S_{t+n},A_{t+n})
\end{eqnarray}$$

- すなわち、状態行動対$(S_t,A_t)$以降の収益の本来のサンプル値
$$\begin{eqnarray}
G_t &=& R_{t+1} + \gamma R_{t+2} + \cdots + \gamma^{T-t+1} R_T
\\ &=& R_{t+1} + \gamma R_{t+2} + \cdots + \gamma^{n-1}R_{t+n} + \gamma^n G_{t+n}
\end{eqnarray}$$
のうち、
 - 状態行動対$(S_{t+1},A_{t+1})から$n-step先までの報酬$R_{t+1}, \cdots, R_{t+n}$を実際にサンプリング
 - それ以降の収益$G_{t+n}$は、時点$t+n$の状態$S_{t+n}$と現状の$Q$関数から推定した「$G_{t+n}$の期待値」$Q(S_{t+n},A_{t+n})$で代替

- 各時点$t$において、状態行動対$(S_t,A_t)$から現状の方策と$Q$関数、未知の環境モデルの下で、$n$ステップ先までサンプリングして$G_t^{(n)}$を得たら、時点$t$に戻ってきて（?）全ての状態行動対$(s,a)$について
$$
Q_{t+1} (s,a) = Q_t(s,a) + \alpha \left( G_t^{(n)} - Q_t(S_t,A_t)\right)1(S_t=s,A_t=a)
$$
により$Q$関数を更新する．


- $n \rightarrow \infty$としたものがMCに対応
- $n = 1$としたものが1stepTDに対応
- 場合によって最適な$n$が異なる
- MCほど待たなくて良いが、オンライン学習ができない$(n \geq 2)$

**TD($\lambda$)法**:

- 目標値を$\lambda$-収益：
$$
G_t^\lambda = (1 - \lambda) \sum_{n=1}^{\infty}\lambda^{n-1}G_t^{(n)}
$$
とする.
 - 各$G_t^{(n)}$を$\lambda^{n-1}$により重みづけして足し上げ($0 \leq \lambda \leq 1$)
 - $1-\lambda$は正規化因子
 $$
 (1 -\lambda) \sum_{n=1}^\infty \lambda^{n-1} G = (1-\lambda) \frac{1}{1 - \lambda} G = G
 $$
- 各時点$t$において、全ての状態行動対$(s,a)$について
$$
Q_{t+1} (s,a) = Q_t(s,a) + \alpha \left( G_t^\lambda - Q_t(S_t,A_t)\right)1(S_t=s,A_t=a)
$$
によって更新.

- TD($\lambda$)誤差：
$$
G_t^{\lambda} - Q_t(S_t,A_t) = (1-\lambda)\sum_{n=1}^\infty \lambda^{n-1}\left(G_t^{(n)} - Q_t(S_t,A_t)\right)
$$

**n-step TD誤差の展開：**

$$\begin{eqnarray}
G_t^{(n)} - Q_t(S_t,A_t) 
&=& \sum_{k=1}^{n}\gamma^{k-1}R_{t+k} + \gamma^n Q_t(S_{t+n}, A_{t+n}) - Q_t(S_t,A_t)
\\ &=& \sum_{k=1}^{n}\gamma^{k-1}\left\{\delta_{t+k} - \gamma Q_{t+k-1}(S_{t+k},A_{t+k}) + Q_{t+k-1}(S_{t+k-1},A_{t+k-1})\right\} + \gamma^n Q_t(S_{t+n}, A_{t+n}) - Q_t(S_t,A_t)
\\ &=& \sum_{k=1}^{n}\gamma^{k-1}\delta_{t+k} - \underbrace{\sum_{k=1}^{n}\gamma^k Q_{t+k-1}(S_{t+k},A_{t+k})}_{\sum_{k=1}^{n-1}\gamma^k Q_{t+k-1}(S_{t+k},A_{t+k}) + \gamma^n Q_{t+n-1}(S_{t+n},A_{t+n})} + \underbrace{\sum_{k=1}^{n}\gamma^{k-1}Q_{t+k-1}(S_{t+k-1},A_{t+k-1})}_{Q_t(S_t,A_t) + \sum_{k=1}^{n-1}\gamma^kQ_{t+k}(S_{t+k},A_{t+k})} + \gamma^n Q_t(S_{t+n}, A_{t+n}) - Q_t(S_t,A_t)
\\ &=& \sum_{k=1}^{n}\gamma^{k-1}\delta_{t+k} +  \sum_{k=1}^{n-1}\gamma^k \left\{Q_{t+k}(S_{t+k},A_{t+k})-Q_{t+k-1}(S_{t+k},A_{t+k}) \right\} + \gamma^n \left\{Q_t(S_{t+n},A_{t+n}) - Q_{t+n-1}(S_{t+n},A_{t+n}) \right\}
\end{eqnarray}$$
- ただし、
$$
\delta_{t+k} = R_{t+k} + \gamma Q_{t+k-1}(S_{t+k},A_{t+k}) - Q_{t+k-1}(S_t,A_t)
$$
は時点$t+k-1$から$t+k$への1stepTD誤差である.
- $Q$関数がステップごとに逐次更新されるときは、第二項、第三項における$Q$関数の差分は$0$にならず、学習率$\alpha$に比例する. 
- $Q$関数をステップごとに更新せずにnステップ分のサンプルを取って計算するときは補正項$O(\alpha)$は現れない.
- よって、$O(\alpha)$を無視する近似において、時点$t$におけるn-stepTD誤差は
$$
G_t^{(n)} - Q_t(S_t,A_t) = \sum_{k=1}^{n}\gamma^{k-1}\delta_{t+k} + O(\alpha)
$$
のように、時点$t$以降のn個の1stepTD誤差の割引級数和として展開できる.

$$\begin{eqnarray}
G_t^{\lambda} - Q_t(S_t,A_t) 
&=& (1-\lambda)\sum_{n=1}^\infty \lambda^{n-1}\left(G_t^{(n)} - Q_t(S_t,A_t)\right) 
\\ &=& (1-\lambda)\sum_{n=1}^\infty \lambda^{n-1}\left( \sum_{k=1}^{n}\gamma^{k-1}\delta_{t+k} + O(\alpha) \right) 
\\ &=& \underbrace{\sum_{n=1}^\infty \sum_{k=1}^{n}(1-\lambda)\lambda^{n-1} \gamma^{k-1}\delta_{t+k}}_{(*)} + O(\alpha)
\end{eqnarray}$$
ここで、任意の2数列$\{a_n\}$、$\{b_m\}$間に成り立つ
$$
\sum_{n=1}^N \sum_{m=1}^n a_n b_m = \sum_{m=1}^N\sum_{n=m}^N a_n b_m
$$
の関係を利用して
$$\begin{eqnarray}
(*) &=& \sum_{k=1}^\infty \sum_{n=k}^\infty(1-\lambda)\lambda^{n-1} \gamma^{k-1}\delta_{t+k}
\\ &=& \sum_{k=1}^{T-t} (1-\lambda) \lambda ^{k-1}\frac{1}{1 -\lambda} \gamma^{k-1}\delta_{t+k}
\\ &=& \sum_{k=1}^\infty (\lambda\gamma)^{k-1}\delta_{t+k} 
\end{eqnarray}$$

よって、TD($\lambda$)誤差は
$$
G_t^{\lambda} - Q_t(S_t,A_t) = \sum_{k=1}^\infty (\lambda\gamma)^{k-1}\delta_{t+k} + O(\alpha)
$$
のように、学習率$\alpha$の一次の誤差を近似し$\lambda\gamma$を割引率とみなしたときの、$k=1,2,\cdots$ステップ先での1stepTD誤差の割引級数和として展開できる.

※ここで、エピソードの終端状態Tで有限にして議論しないと話が進まなくなったので、教科書の$\lambda$-収益の定義から出発してもう一度同じような証明をやった上でこの続きに入っていきます.

**終端状態Tを考慮するとき:**

- $\lambda$-収益の定義式：
$$
G_t^\lambda = (1 - \lambda) \sum_{n=1}^{\infty}\lambda^{n-1}G_t^{(n)}
$$
について、終端状態Tを考慮して次のように展開できる：
$$\begin{eqnarray}
(右辺)&=& (1 - \lambda) \sum_{n=1}^{T-t}\lambda^{n-1}G_t^{(n)} + \underbrace{(1 - \lambda) \sum_{n=T-t+1}^{\infty}\lambda^{n-1}G_t^{(n)}}_{(*)} \\
(*) &=& (1 - \lambda)\sum_{n=T-t+1}^{\infty}\lambda^{n-1}G_t^{(n)}
\\ &=& (1 - \lambda) \lambda^{T-t}\sum_{n=1}^{\infty}\lambda^{n-1}G_t^{(T-t+n)}
\\ &=& (1 - \lambda) \lambda^{T-t}\underbrace{\sum_{n=1}^{\infty}\lambda^{n-1}}_{1 / (1 - \lambda)}G_t^{(T-t)}
\\ &=& \lambda^{T-t} G_t^{(T-t)}
\end{eqnarray}$$
ただし、終端状態T以降の任意の時点$T+k$における報酬$R_{T+k}$は0なので
$$G_t^{(T-t+n)}=G_t^{(T-t)} \quad(n=1,2,\cdots)$$
であることを用いた.
- 以上より、
$$
G_t^{\lambda} = (1 - \lambda) \sum_{n=1}^{T-t}\lambda^{n-1}G_t^{(n)} + \lambda^{T-t} G_t^{(T-t)}
$$

- この両辺から
$$
Q_t(S_t,A_t) = \left\{ (1-\lambda)\sum_{n=1}^{T-t} \lambda^{n-1} + \lambda^{T-t} \right\} Q_t(S_t,A_t)
$$
を引いて

$$
G_t^{\lambda} - Q_t(S_t,A_t) = (1-\lambda)\sum_{n=1}^{T-t} \lambda^{n-1}\left(G_t^{(n)} - Q_t(S_t,A_t)\right) + \lambda^{T-t}\left(G_t^{(T-t)} - Q_t(S_t,A_t) \right)
$$

$$\begin{eqnarray}
G_t^{\lambda} - Q_t(S_t,A_t) 
&=& (1-\lambda)\sum_{n=1}^{T-t} \lambda^{n-1}\left(G_t^{(n)} - Q_t(S_t,A_t)\right) + \lambda^{T-t}\left(G_t^{(T-t)} - Q_t(S_t,A_t) \right)
\\ &=& (1-\lambda)\sum_{n=1}^{T-t} \lambda^{n-1}\left( \sum_{k=1}^{n}\gamma^{k-1}\delta_{t+k} + O(\alpha) \right) + \lambda^{T-t}\left( \sum_{k=1}^{T-t}\gamma^{k-1}\delta_{t+k} + O(\alpha) \right)
\\ &=& \underbrace{\sum_{n=1}^{T-t} \sum_{k=1}^{n}(1-\lambda)\lambda^{n-1} \gamma^{k-1}\delta_{t+k}}_{(*)} + \lambda^{T-t} \sum_{k=1}^{T-t}\gamma^{k-1}\delta_{t+k} + O(\alpha)
\end{eqnarray}$$
- ここで、任意の2数列$\{a_n\}$、$\{b_m\}$間に成り立つ
$$
\sum_{n=1}^N \sum_{m=1}^n a_n b_m = \sum_{m=1}^N\sum_{n=m}^N a_n b_m
$$
の関係を利用して
$$\begin{eqnarray}
(*) &=& \sum_{k=1}^{T-t} \sum_{n=k}^{T-t}(1-\lambda)\lambda^{n-1} \gamma^{k-1}\delta_{t+k}
\\ &=& \sum_{k=1}^{T-t} (1-\lambda) \frac{\lambda^{k-1} - \lambda^{T-t}}{1 -\lambda} \gamma^{k-1}\delta_{t+k}
\\ &=& \sum_{k=1}^{T-t} (\lambda^{k-1} - \lambda^{T-t})\gamma^{k-1}\delta_{t+k}
\\ &=& \sum_{k=1}^{T-t} \lambda^{k-1}\gamma^{k-1}\delta_{t+k} - \lambda^{T-t}\sum_{k=1}^{T-t}\gamma^{k-1}\delta_{t+k}
\end{eqnarray}$$
と変形できる.　
- よって、TD($\lambda$)誤差は
$$
G_t^{\lambda} - Q_t(S_t,A_t) = \sum_{k=1}^{T-t} (\lambda\gamma)^{k-1}\delta_{t+k} + O(\alpha)
$$
のように、学習率$\alpha$の一次の誤差を近似し$\lambda\gamma$を割引率とみなしたときの、$k=1,2,\cdots,T-t$ステップ先での1stepTD誤差の割引級数和として展開できる.

以上より、1エピソードにおける各状態行動対$(s,a)$についてのTD($\lambda$)誤差の総和は
$$\begin{eqnarray}
\sum_{t=0}^{T-1}\left(G_t^{\lambda} - Q_t(S_t,A_t)\right)1(S_t=s,A_t=a) 
&=& \sum_{t=0}^{T-1}\left(\sum_{k=1}^{T-t} (\lambda\gamma)^{k-1}\delta_{t+k}+ O(\alpha) \right)1(S_t=s,A_t=a) \\
(右辺) &=& \left( \sum_{t=0}^{T-1}\sum_{k=1}^{T-t} (\lambda\gamma)^{k-1}\delta_{t+k}+ O(\alpha) \right)1(S_t=s,A_t=a)
\\ &=& \sum_{t=0}^{T-1}\sum_{k=0}^{T-1-t} (\lambda\gamma)^k\delta_{t+k+1} 1(S_t=s,A_t=a) + O(\alpha)
\\ &=& \sum_{t=0}^{T-1}\sum_{k=t}^{T-1} (\lambda\gamma)^{k-t}\delta_{k+1} 1(S_t=s,A_t=a) + O(\alpha)
\\ &=& \sum_{k=0}^{T-1} \delta_{k+1} \sum_{t=0}^{k} (\lambda\gamma)^{k-t} 1(S_t=s,A_t=a) + O(\alpha)
\\ &=& \sum_{k=0}^{T-1} \delta_{k+1} E_{k+1}(s,a) + O(\alpha) \\
\therefore \sum_{t=0}^{T-1}\left(G_t^{\lambda} - Q_t(S_t,A_t)\right)1(S_t=s,A_t=a) 
&=& \sum_{k=0}^{T-1} \delta_{k+1} E_{k+1}(s,a) + O(\alpha)
\end{eqnarray}$$
- ただし
$$\begin{eqnarray}
E_{k+1}(s,a) &=& \sum_{t=0}^{k} (\lambda \gamma)^{k-t} 1(S_t=s,A_t=a)
\\ &=& 1(S_k=s,A_k=a) + (\lambda \gamma)^{1} 1(S_{k-1}=s,A_{k-1}=a) + \cdots + (\lambda \gamma)^{k} 1(S_0=s,A_0=a)
\end{eqnarray}$$

- 以上より、TD($\lambda$)法において、1つのエピソード内で、各状態行動対$(s,a)$について、時点$t$から$t+1$へのステップにおける$Q$関数の変化量$\Delta Q_{t+1}(s,a)$について
$$
\frac{1}{\alpha}\Delta Q_{t+1}(s,a) = \left(G_t^{\lambda} - Q_t(S_t,A_t)\right)1(S_t=s,A_t=a) = \delta_{k+1} E_{k+1}(s,a) + O(\alpha)
$$
が成り立つ.
- このことより、各時点$t$において全ての状態行動対$(s,a)$について
$$\begin{eqnarray}
Q_{t+1}(s,a) &=& Q_t(s,a) + \alpha \left(G_t^{\lambda} - Q_t(S_t,A_t)\right)1(S_t=s,A_t=a)
\\ &=&  Q_t(s,a) + \alpha \left( \sum_{k=1}^{T-t} (\lambda\gamma)^{k-1}\delta_{t+k} \right) 1(S_t=s,A_t=a)
\\ &=& Q_t(s,a) + \alpha \delta_{t+1} E_{t+1}(s,a) 
\\ E_{t+1}(s,a) &=& \lambda \gamma E_t(s,a) + 1(S_t=s,A_t=a)
\\ \delta_{t+1} &=& R_{t+1} + \gamma Q_t(S_{t+1},A_{t+1}) - Q_t(S_t,A_t)
\end{eqnarray}$$
のように更新することで、近似的にオンライン学習が可能になる.

- TD($\lambda$)についての意味の解釈は、個人的に、https://yamaimo.hatenablog.jp/entry/2015/12/12/200000 というブログが参考になりました.

$\epsilon$-greedy法:
- 環境モデルが未知で、$Q$関数が推定値にすいぎないとき、greedyな方策は必ずしも最適にはならない：
$$
a_{\pi,s}^* \neq \mbox{argmax}_a Q(s,a)
$$
- 確率的なばらつきによって、最適行動の価値関数が過小評価されている可能性がある．
- 確率$\epsilon$で無作為に行動を選択し、確率$1-\epsilon$でgreedy法により行動を選択する.
$$
\pi(a|s) = 
\left\{\begin{array}{ll}
(1 - \epsilon)/|A^*(s)| + \epsilon/|A(s)|, & a \in A^*(s) \\
\epsilon/|A(s)|, & \mbox{otherwise}
\end{array}\right.\\
 A^*(s) = \left\{ a^* \in A(s)| a^* = \mbox{argmax}_a Q(s,a)\right\}
$$

SARSA:
- $\epsilon$-greedy法により方策改善し、その方策に基いて時点$t+1$の行動$A_{t+1}$を生成しながら(0)TDにより$Q$関数を学習していく方法.
- サンプリングが$(S_t, A_t, R_{t+1},S_{t+1},A_{t+1})$であることからSARSA

Q学習:
- TD(0)法のTD誤差の次時点の$Q$関数の最大値を選ぶようにする.
$$\begin{eqnarray}
\delta_{t+1} &=& R_{t+1} + \gamma \mbox{max}_{a'}Q_t(S_{t+1},a') - Q_t(S_{t},A_{t}) \\ Q_{t+1}(s,a) &=& Q_t(s,a) + \alpha \delta_{t+1}1(S_t=s,A_t=a)
\end{eqnarray}$$
- TD誤差の推定における次行動の選択の仕方（推定方策）がgreedy
- 方策の改善の仕方は$\epsilon$-greedy
- 方策オフ型：推定方策として挙動方策とは異なる方策を採用する学習方法

DQN (Double Q Network):
- 各ステップで更新していく$Q$関数($Q_{main}$)とtarget値の推定に使う$Q$関数($Q_{target}$)を別々に用意して行う$Q$学習.
- 一定ステップ毎に$Q_{target}$を$Q_{main}$と同期させて、目標値の推定値を更新.
- $Q$関数はニューラルネットワークでモデル化：
$$ Q: s \in S \rightarrow  x(s) \in R^m \rightarrow Q(s) = (Q(s,a_1), Q(s,a_2), \cdots, Q(s,a_n))^T \in R^n$$
 - ひとつの状態$s$に対して取り得る全ての行動$a \in A$の行動価値関数$Q(s,a)$を全て一度に返すモデルにすることで計算コスト減.
 - target値内の行動価値関数の推定値は$\mbox{max}_{a'}Q_{target}(s,a') = \mbox{max}(Q_{target}(s))$ですぐに求められる.
 - 各ステップの$Q_{main}$の更新式は
 $$
 \delta_{t+1} = R_{t+1} + \mbox{max}_{A_{t+1}}Q_{target}(S_{t+1},A_{t+1}) - Q_{main}(S_t,A_t) \\
 \left( 
   \begin{array}{c} Q_{main}(s,a_1) \\ Q_{main}(s,a_2) \\ \cdots \\ Q_{main}(s,a_n)
   \end{array}\right) \leftarrow \left(\begin{array}{c} Q_{main}(s,a_1) \\ Q_{main}(s,a_2) \\ \cdots \\ Q_{main}(s,a_n) \end{array}\right) + \alpha \delta_{t+1}\left( 
   \begin{array}{c} 1(S_t=s,A_t=a_1) \\ 1(S_t=s,A_t=a_2) \\ \cdots \\ 1(S_t=s,A_t=a_n)
   \end{array}\right)
$$
と書ける.

 - 一定ステップごとに$Q_{target} \leftarrow Q_{main}$により$Q_{target}$を更新.

- $Q_{target}$を使うメリット：
- reward clipping
- 

以上にみてきたように、行動価値関数の推定値$Q(s,a)$のテーブルを用意して推定していく方法は、状態空間や行動空間が高次元または連続になると破綻する.　そこで、次脳ような方策ベース手法を導入する：

方策ベース手法：
- 方策をパラメータ$\theta$を用いて推定（モデル化）し、目的関数$J(\theta)$に基づき$\theta$を改善していくことで方策を最適化していく:
$$\begin{align}
\pi(a|s,\theta) \\
\theta \leftarrow \theta + \alpha \nabla_\theta J(\theta)
\end{align}$$　


ベルマン方程式
$$
\begin{align}
v_\pi(s) = \sum_{a \in A} \pi(a|s) q_\pi(s,a)
\end{align}
$$
は方策$\pi$を介して$\theta$にのみ依存するとするとして両辺を$\theta$で微分して整理すると
$$\begin{eqnarray}
\nabla_\theta v_\pi(s) &=& \sum_{s' \in S} d^\pi(s,s') \sum_{a \in A} (\nabla_\theta \pi(a|s',\theta)) q_\pi(s',a)
\end{eqnarray}$$
を得る（詳細は教科書参照）.　ただし、
$$\begin{eqnarray}
\\ d^\pi(s,s') &=& \sum_{k=0}^\infty \gamma^k [(P^\pi )^k]_{ss'},
\\ [P^\pi]_{ss'} &=& P_{ss'}^\pi = \sum_{a \in A} \pi(a|s,\theta) p(s'|s,a).
\end{eqnarray}$$
ここで
$$
\nabla_x \log(f(x)) = \sum_{i} \frac{\partial \log(f(x))}{\partial x_i}
= \sum_{i} \frac{\partial \log(f(x))}{\partial f(x)}\frac{\partial f(x)}{\partial x_i} = \frac{1}{f(x)} \nabla_x f(x), \\
\nabla_x f(x) = f(x) \nabla_x \log (f(x))
$$
を利用して
$$
\nabla_\theta v_\pi(s_0) = \sum_{s \in S} d^\pi(s_0,s) \sum_{a \in A} \pi(a|s,\theta)(\nabla_\theta \log \left(\pi(a|s,\theta))\right) q_\pi(s,a)
$$
とできる.　したがって、目的関数を次のように設定して方策勾配を得られる.　
$$\begin{eqnarray}
J(\theta) &=& v_\pi(s_0),
\\ \nabla_\theta J(\theta) &=& E_\pi[\left( \nabla_\theta \log(\pi(a|s,\theta))\right) q_\pi(s,a)]
\end{eqnarray}$$

ベースライン関数の導入：
- 状態$s$に依存し、行動$a$に依存しない任意の関数$b(s)$について次が成り立つ:
$$\begin{eqnarray}
E_\pi[\left( \nabla_\theta \log(\pi(a|s,\theta))\right) b(s)]
&=& \sum_{s \in S} d^\pi(s_0,s) \sum_{a \in A} \pi(a|s,\theta)(\nabla_\theta \log \left(\pi(a|s,\theta))\right) b(s)
\\ &=& \sum_{s \in S} d^\pi(s_0,s) b(s) \sum_{a \in A} \nabla_\theta \pi(a|s,\theta )
\\ &=& \sum_{s \in S} d^\pi(s_0,s) b(s) \nabla_\theta \underbrace{\sum_{a \in A} \pi(a|s,\theta )}_{1:定数}
\\ &=& 0
\end{eqnarray}$$
よって
$$
E_\pi[\left( \nabla_\theta \log(\pi(a|s,\theta))\right) q_\pi(s,a)] = E_\pi[\left( \nabla_\theta \log(\pi(a|s,\theta))\right) (q_\pi(s,a) - b(s))]
$$

- ベースライン関数として価値関数$v_\pi(s)$を選び、行動価値関数を価値関数でシフトした関数:
$$
a_\pi(s,a) = q_\pi(s,a) - v_\pi(s)
$$
をアドバンテージ関数と呼ぶ. これは、方策$\pi$、状態$s$のもとで行動$a$が平均と比べてどのくらい価値が高いかを表している.
- 方策勾配定理における行動価値関数をアドバンテージ関数で置き換えるとバリアンスが低く抑えられた方策勾配が得られる:(?)
$$\begin{eqnarray}
\nabla_\theta J(\theta) &=& E_\pi[\left( \nabla_\theta \log(\pi(a|s,\theta))\right) (q_\pi(s,a) - v_\pi(s))]
\\ & \approx & \frac{1}{T} \sum_{t=0}^{T-1} \nabla_\theta (\log (\pi(A_t|S_t,\theta)))(Q(S_t,A_t) - V(S_t))
\end{eqnarray}$$

- 以上の計算過程をまとめると、
$$\begin{eqnarray}
\nabla_\theta J(\theta) 
&=& \nabla_\theta v_\pi(s)
\\ &=& \sum_{s \in S} d^\pi(s_0,s) \sum_{a \in A} \pi(a|s,\theta)(\nabla_\theta \log \left(\pi(a|s,\theta))\right) q_\pi(s,a)
\\ &=& \sum_{s \in S} d^\pi(s_0,s) \sum_{a \in A} \pi(a|s,\theta)(\nabla_\theta \log \left(\pi(a|s,\theta))\right) (q_\pi(s,a) - v_\pi(s))
\\ &=& E_\pi[\left( \nabla_\theta \log(\pi(a|s,\theta))\right) (q_\pi(s,a) - v_\pi(s))]
\\ & \approx & \frac{1}{T} \sum_{t=0}^{T-1} \nabla_\theta (\log (\pi(A_t|S_t,\theta)))(Q(S_t,A_t) - V(S_t))
\end{eqnarray}$$

- 問題となるのは、方策ベースを使用するような高次元または連続な行動空間において、$Q$テーブルの使用は非現実的.
- 方策改善のための方策のモデル化に加えて、方策評価のための行動価値関数$Q$のモデル化も必要になってくる.
- 行動価値関数の代わりに価値関数をモデル化:$V_\omega(s)$

Actor-Critic法:
- agentが担う方策改善と方策評価の機能を分離し、個々をモデル化する方法
 - Actor: 方策$\pi(a|s,\theta)$をモデル化し、方策勾配に基づきパラメータ$\theta$を更新(=方策の改善)
 - Critic: 価値関数$V_\omega(s)$をモデル化し、TD誤差に基づきパラメータ$\omega$を更新(=価値関数の推定精度の向上)

Actor:
- 目的関数の設定:
 - 方策を最適化したい
 - 方策勾配を上っていった頂上に着きたい
 - 最小化すべき目的関数は、$\theta$で微分したら方策勾配の$-1$倍になるもの:
 $$
 L_{actic}(\theta) = - J(\theta) \approx - \frac{1}{T}\sum_{t=0}^{T-1}(\log\pi(A_t|S_t,\theta)) (Q(S_t,A_t) - V_\omega(S_t)) $$
 - このままでは、高次元または連続な行動空間においてやっかいな$Q$を含んでいる.
 そこで、
 $$
 \delta(s,r,s') = r + \gamma v_\pi(s') - v_\pi(s)
 $$
 の期待値は
 $$\begin{eqnarray}
E_\pi[\delta(s,r,s')] &=& E_\pi[r + \gamma v_\pi(s') - v_\pi(s)]
\\ &=& E_\pi[q_\pi(s,a) - v_\pi(s)]
 \end{eqnarray}$$
 となるという事実を使って$Q$を消去する.　
 
 - すなわち、
 $$ Q(S_t,A_t) - V_\omega(S_t) $$
 の期待値は
 $$ \delta_{t+1}(\omega) = R_{t+1} + \gamma V_\omega(S_{t+1}) - V_\omega(S_{t}) $$
 の期待値と等しくなることを利用して
 $$ Q(S_t,A_t) - V_\omega(S_t) \approx \delta_{t+1}(\omega)$$
 でとして
 $$
 L_{actic}(\theta) = - J(\theta) \approx - \frac{1}{T}\sum_{t=0}^{T-1}(\log\pi(A_t|S_t,\theta)) \delta_{t+1}(\omega) $$
 と近似的にかける.　

 これにより、高次元または連続な行動空間では発散してしまう$Q(S_t,A_t)$を使わずに方策勾配を求められるようになった.

Critic:
- 目的関数の設定　
 - 最小化すべき目的関数はTD誤差の二乗和:
 $$
 L_{critic} = \sum_{t=0}^{T-1}|\delta_{t+1}(w)|^2,\\
 \delta_{t+1} = R_{t+1} + \gamma V_\omega (S_{t+1}) - V_\omega(S_t)
 $$

教科書にあったこのような記述だけではCriticの目的関数の背景が良く分からなかったため、以下にDavid Silver氏によるRL Course - Lecture 6のValue Function Approximationの説明を参考にまとめました：

価値関数モデルの更新、すなわち、パラメータ$w$の更新：
- 目的関数：平均二乗誤差（教科書とは正負が逆になっていることに注意.　こちらでは目的関数を最大化するように書くことで、結果の式が上の方で述べた価値関数推定値の更新式を一般化できるようにまとめてある.）
$$
L(w) = - \frac{1}{2} E_\pi[(v_\pi(s) - \hat v (s,w))^2]
$$

$$
\nabla_w L(w)  = (v_\pi(s) - \hat v (s,w)) \nabla_w \hat v(s,w)
$$
- よって、パラメータ$w$の更新式は:
$$\begin{eqnarray}
w &\leftarrow& w + \alpha \nabla_w L(w) \\
  &=& w + \alpha (v_\pi(s) - \hat v (s,w)) \nabla_w \hat v(s,w) \\
  &=& w + \alpha \delta (w) \nabla_w \hat v(s,w) \\
\delta (w) &=& v_\pi(s) - \hat v (s,w)
\end{eqnarray}$$

- ただし、予測を行う際には当然、価値関数の真の値$v_\pi(s)$は分からないので、他の目標値で代替：
 - $G_t = R_{t+1} + \gamma R_{t+2} + \cdots + \gamma^{T-t-1} R_{T}$
 - $G_t^{(1)} = R_{t+1} + \hat v (s,w)$
   - $\delta (w) = \delta_{t+1}(w) = R_{t+1} + \gamma \hat v(S_{t+1},w) - v(S_t,w)$ ：1stepTD誤差
 - $G_t^{(n)} = R_{t+1} + \gamma R_{t+2} + \cdots + \gamma^{n-1} R_{t+n} + \gamma^n \hat v(s,w)$
   - $\delta(w) = \delta_{t+1}^{(n)}(w) = \delta_{t+1}(w) + \gamma \delta_{t+2}(w) + \cdots + \gamma^{n-1} \delta_{t+n}(w)$：n-stepTD誤差
   - $\delta_{t+k}(w) = R_{t+k} + \gamma \hat v (S_{t+k},w) - \hat v (S_{t+k-1},w)$
 - $G_t^\lambda = (1 - \lambda) \sum_{n=1}^\infty \lambda^{n-1} G_t^{(n)}$
   - $\delta(w) = \delta_{t+1}^\lambda (w) = \sum_{k=1}^{T-t} (\lambda \gamma)^{k-1}\delta_{t+k}(w)$：TD($\lambda$)誤差(forward)

- このうち、目標値に$G_t^{(1)}$を用いているのが教科書のCritic.

 

以下では、価値関数の近似方法の主要な手段（線形近似、ニューラルネットワークモデル）について簡単にまとめました.

価値関数の線形モデル：
- 価値関数を線形モデルで近似すると、次のように表せる：
$$\begin{eqnarray}
x(s) &=& (x_1(s),x_2(s), \cdots, x_n(s))^T \\
w &=& (w_1,w_2, \cdots, w_n)^T \\
\hat v(s,w) &=& w^Tx(s) = \sum_{i=1}^n w_i x_i(s)
\end{eqnarray}$$
- よって、線形モデルにおいては
$$\nabla_w \hat v(s,w) = x(s)$$
よって更新式は $$w \leftarrow w + \alpha (v_\pi(s) - \hat v (s,w))x(s)$$となる.
- table  lookupは価値関数線形近似の特別な場合である:
$$
 x(s) = \left( 
   \begin{array}{c} 1(s=s_1) \\ 1(s=s_2) \\ \cdots \\ 1(s=s_n)
   \end{array}\right) \\
   \hat v (s,w) = \left( 
   \begin{array}{c} 1(s=s_1) \\ 1(s=s_2) \\ \cdots \\ 1(s=s_n)
   \end{array}\right) \cdot \left( 
   \begin{array}{c} w_1 \\ w_2 \\ \cdots \\ w_n
   \end{array}\right)
$$
$w_1, w_2, \cdots,w_n$はそれぞれ$V(s_1),V(s_2),\cdots,V(s_n)$を表す.
- よって更新式は：
$$
\left( 
   \begin{array}{c} V(s_1) \\ V(s_2) \\ \cdots \\ V(s_n)
   \end{array}\right) \leftarrow \left(\begin{array}{c} V(s_1) \\ V(s_2) \\ \cdots \\ V(s_n) \end{array}\right) + \alpha (v_\pi(s) - \hat v (s,w))\left( 
   \begin{array}{c} 1(s=s_1) \\ 1(s=s_2) \\ \cdots \\ 1(s=s_n)
   \end{array}\right)
$$
とまとめられる.


価値観数のニューラルネットワークモデル：
- ニューラルネットワークのパラメータ全体をまとめて$w$とすると:
$$
\hat v(s,w) = \mbox{Model}_w^{NN}(x(s))
$$
状態$s$の特徴量は$x(s) \in R^n$で表され、それがニューラルネットワークモデル$y= \mbox{Model}_w^{NN}(x)$に放り込まれて、状態$s$の価値関数の推定値$\hat v (s,w) = y$が返される.

- ニューラルネットワークモデルのパラメータの更新は、上で述べたような更新式で行う.　すなわち、目的関数$L$のパラメータ$w$による勾配$\nabla_w L$に応じ、目的関数を最大化（誤差を小さく）するように、ステップ幅$\alpha$で$w$を更新.

- 例えば、DeepMind社によるインベーダーゲームなどAtariゲームの強化学習では、関数近似に畳み込みニューラルネットワークCNNを用いることで、学習を重ねるうちにネットワークがゲーム画面（状態の生情報）から特徴を自動的にうまく抽出できるようになって、その特徴に基づいて価値関数を推定するようになっていくため上手く学習できるのだと考えられます.